## Remove the effect of the age to see if evereything is driven from thee age

In [1]:
cd ../pipelines

/Users/pldelacour/Documents/PL_Ecole/data_sc_lab/data_sc_lab/task_01/pipelines


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipynb.fs.full.tool_functions import *
from yellowbrick.regressor import AlphaSelection
from  sklearn.metrics import mean_squared_error

/Users/pldelacour/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
df_beha = pd.read_csv('../../../data/Behavioral/AllData.csv')#, nrows=1000)

df_psd_cluster = pd.read_csv("../../../data/EEG/RestingEEG_PSD_Cluster.csv") #, nrows=1000)
df_psd_channel = pd.read_csv("../../../data/EEG/RestingEEG_PSD_Channel.csv") #, nrows=5)
df_spectro_cluster = pd.read_csv("../../../data/EEG/RestingEEG_Spectro_Cluster.csv") #, nrows=1000)
df_spectro_channel = pd.read_csv("../../../data/EEG/RestingEEG_Spectro_Channel.csv") #, nrows=5)

df_microstate = pd.read_csv("../../../data/EEG/RestingEEG_Microstates.csv") #, nrows=5)

/Users/pldelacour/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (66,68,80,81,82,83,84,95,96,97,98,99,110,111,112,113,114,125,126,127,128,129,140,141,142,144,155,156,157,159,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,11

In [4]:
# Extract relevant info from the Behavioral data 
y_labels = df_beha[["Patient_ID", "Age", "DX_01_Cat"]]
y_labels_healthy = y_labels[y_labels["DX_01_Cat"].str.contains("No Diagnosis Given") == True]

y_labels = y_labels[["Patient_ID", "Age"]]
y_labels_healthy = y_labels_healthy[["Patient_ID", "Age"]]
y_labels_healthy

,Patient_ID,Age
9,NDARCF462HNM,14.216632
11,NDARFC233RND,9.153661
22,NDARDT800YVF,17.510381
25,NDARZT011LBZ,7.760209
26,NDARKV482FU9,10.977298
...,...,...
2088,NDARAV069HGT,13.896189
2089,NDARZC497BFU,17.419917
2090,NDARDZ737LG9,13.031143
2094,NDARRT283RHB,10.712069


In [5]:
df_psd_cluster.rename(columns={'id': 'Patient_ID'}, inplace=True)
df_spectro_cluster.rename(columns={'id': 'Patient_ID'}, inplace=True)
df_microstate.rename(columns={'id': 'Patient_ID'}, inplace=True)
print("Original lengths: " + str(len(df_psd_cluster)) + "(psd), " + \
     str(len(df_spectro_cluster)) + "(spectro), " + str(len(df_microstate)) + "(micro)")

# Aggregate with label
df_label_psd_cluster = pd.merge(y_labels_healthy, df_psd_cluster, on='Patient_ID')
df_label_spectro_cluster = pd.merge(y_labels_healthy, df_spectro_cluster, on='Patient_ID')
df_label_microstate = pd.merge(y_labels_healthy, df_microstate, on='Patient_ID')
print("Only healthy people, resulting lengths: " + str(len(df_label_psd_cluster)) + "(psd), " + \
     str(len(df_label_spectro_cluster)) + "(spectro), " + str(len(df_label_microstate)) + "(micro)")

# Drop NaN values
df_label_psd_cluster = fill_with_median(df_label_psd_cluster)
df_label_spectro_cluster = fill_with_median(df_label_spectro_cluster)
df_label_microstate = fill_with_median(df_label_microstate)
print("Replace NaN by median, resulting lenghts: " + str(len(df_label_psd_cluster)) + "(psd), " + \
     str(len(df_label_spectro_cluster)) + "(spectro), " + str(len(df_label_microstate)) + "(micro)")

Original lengths: 1485(psd), 1485(spectro), 1108(micro)
Only healthy people, resulting lengths: 174(psd), 174(spectro), 128(micro)
Replace NaN by median, resulting lenghts: 174(psd), 174(spectro), 128(micro)


In [6]:
#from sklearn.model_selection import train_test_split

In [7]:
test_ratio = 0.2

In [8]:

# Define Train and Test data 
print()
print("---- Split into Train and Test data ----")
print()
(X_train_psd, y_train_psd, X_test_psd, y_test_psd) = \
                                                    train_test_split(df_label_psd_cluster, test_ratio)
(X_train_spe, y_train_spe, X_test_spe, y_test_spe) = \
                                                    train_test_split(df_label_spectro_cluster, test_ratio)
(X_train_mic, y_train_mic, X_test_mic, y_test_mic) = \
                                                    train_test_split(df_label_microstate, test_ratio)

print("Test lengths: " + str(len(y_test_psd)) + "(psd), " + \
     str(len(y_test_spe)) + "(spectro), " + str(len(y_test_mic)) + "(micro)")

# Feature selection by correlation
print()
print("---- Feature selection by correlation ----")
print()


---- Split into Train and Test data ----

Test lengths: 34(psd), 34(spectro), 25(micro)

---- Feature selection by correlation ----



In [9]:
x_train = X_train_spe.loc[:,X_train_spe.columns != 'Patient_ID']
y_train = y_train_spe.loc[:,y_train_spe.columns != 'Patient_ID']
x_test = X_test_spe.loc[:,X_train_spe.columns != 'Patient_ID']
y_test = y_test_spe.loc[:,y_train_spe.columns != 'Patient_ID']

## Remove outliers

In [10]:
 
forest = IsolationForest()
forest.fit(x_train)

# Outlier indices for training
outliers_training = forest.predict(x_train)
outliers_training_indices = np.argwhere(outliers_training == 1).flatten()

# Drop signal outliers in training data
x_train = x_train[outliers_training == 1]
y_train = y_train[outliers_training == 1]

print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

(128, 300)
(128, 1)
(34, 300)
(34, 1)


## Scaling 

In [11]:
# Feature Scaling
scaler = StandardScaler()
scaler.fit(x_train)
x_train = pd.DataFrame(scaler.transform(x_train), columns=x_train.columns, index=x_train.index)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns, index=x_test.index)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(128, 300)
(128, 1)
(34, 300)
(34, 1)


## Remove the effect of the age

In [17]:
from sklearn.linear_model import LinearRegression

In [22]:
a = np.ones(x_train.shape)
a.shape

(128, 300)

In [36]:
age =  y_train
x_train_r = np.ones(x_train.shape)
x_test_r = np.ones(x_test.shape)
for i in range(len(x_train.columns)) : 
    fi = x_train.loc[:,x_train.columns[i]]
# age ==> fi 
    reg = LinearRegression().fit(age,fi)
    x_train_r[:,i] = fi - reg.predict(age)
    x_test_r[:, i] = x_test.loc[: , x_test.columns[i]] - reg.predict(y_test)
x_train_r = pd.DataFrame(x_train_r , columns=x_train.columns)
x_test_r = pd.DataFrame(x_test_r, columns= x_test.columns)

In [37]:
print(x_train_r.shape)
print(x_test_r.shape)


(128, 300)
(34, 300)


In [38]:
x_train_r

,eyesclosed_fband_delta_absmean_lfront,eyesclosed_fband_delta_absmean_mfront,eyesclosed_fband_delta_absmean_rfront,eyesclosed_fband_delta_absmean_lpari,eyesclosed_fband_delta_absmean_mpari,eyesclosed_fband_delta_absmean_rpari,eyesclosed_fband_delta_relmean_lfront,eyesclosed_fband_delta_relmean_mfront,eyesclosed_fband_delta_relmean_rfront,eyesclosed_fband_delta_relmean_lpari,...,eyesopen_fooof_peak_freq_rfront,eyesopen_fooof_peak_freq_lpari,eyesopen_fooof_peak_freq_mpari,eyesopen_fooof_peak_freq_rpari,eyesopen_fooof_peak_amplitude_lfront,eyesopen_fooof_peak_amplitude_mfront,eyesopen_fooof_peak_amplitude_rfront,eyesopen_fooof_peak_amplitude_lpari,eyesopen_fooof_peak_amplitude_mpari,eyesopen_fooof_peak_amplitude_rpari
0,1.618501,1.562645,0.115366,-0.652221,-0.520747,-0.044453,0.738020,0.539932,0.720484,1.080689,...,0.402162,0.372036,0.783913,0.249521,0.996117,1.227489,1.390996,0.468662,0.938694,1.094848
1,0.670564,-0.106410,0.541899,0.783344,0.083900,1.059235,-0.753106,-1.189736,-1.036442,-1.123884,...,0.079399,-0.034601,0.083542,-0.087616,1.531317,2.044663,1.674591,1.593461,1.198120,1.321209
2,1.411959,2.093211,1.761507,1.360998,1.668898,2.334024,0.162620,0.549394,0.488482,-0.483793,...,0.483559,-0.469291,-0.523111,0.222488,-0.852542,-0.213554,-0.277233,-1.014403,-0.800685,0.234861
3,-0.254350,-0.827092,-1.038713,-1.279544,-0.973187,-1.014873,1.130858,1.058288,0.661491,0.742611,...,0.050622,0.028984,0.336208,-0.225361,1.799739,-1.154274,-1.055148,-1.192800,-1.007431,-1.108012
4,-0.081744,-0.529874,0.119298,0.886053,0.722528,1.590992,0.455295,0.409462,0.367952,0.153291,...,-0.855538,-0.076218,0.211552,-0.229907,-0.419288,-0.614456,-0.868372,0.019153,0.090044,0.556812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,-0.602959,-0.160488,0.730431,-0.351709,-0.319610,-0.069910,-0.905870,-0.670058,-0.235751,-0.769733,...,0.455960,0.543029,1.045989,0.257004,1.337790,1.628624,1.300206,1.332920,1.076932,1.341602
124,1.496912,1.089632,1.409511,1.319258,0.872234,2.161937,0.009909,0.020123,-0.090136,-0.225935,...,0.570296,0.651652,0.806376,0.233231,-0.112625,0.013922,0.379382,0.099338,0.331887,0.235453
125,1.696742,1.275561,1.133644,1.588922,1.650385,2.161546,-0.747799,-0.790609,-0.661795,-0.566412,...,0.323335,0.288925,0.532788,0.169045,1.111294,2.262518,2.078717,1.810300,1.852879,1.023499
126,-1.231688,-1.763989,-1.017979,-1.677037,-1.686739,-1.684603,0.313183,0.027452,0.380525,0.478709,...,-0.083386,-0.741065,-0.688974,5.767218,-0.114952,-0.193782,-0.047150,0.027888,-0.291632,-0.791304
